In [7]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")


Используемое устройство: cuda


In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Пример загрузки малой части данных
# Загрузка первых 10000 строк для примера 
video_data = pd.read_parquet('video_stat.parquet')
log_data = pd.read_parquet('logs_df_2024-08-05.parquet', engine='pyarrow')

# Используем малую часть данных для ускорения процесса
log_data = log_data.sample(10000, random_state=42)  # Подгружаем случайную часть данных

# Проверка и замена пропущенных значений
video_data = video_data.dropna(subset=['video_id'])  # Удаляем строки с пропущенными 'video_id'
log_data = log_data.dropna(subset=['video_id', 'user_id'])  # Удаляем строки с пропущенными 'video_id' и 'user_id'

# Кодирование идентификаторов видео и пользователей
video_encoder = LabelEncoder()
user_encoder = LabelEncoder()

# Кодирование данных
video_data['video_id_encoded'] = video_encoder.fit_transform(video_data['video_id'])

# Проверка и корректировка: удаление значений, отсутствующих в обучающем наборе
log_data = log_data[log_data['video_id'].isin(video_data['video_id'])]

log_data['video_id_encoded'] = video_encoder.transform(log_data['video_id'])
log_data['user_id_encoded'] = user_encoder.fit_transform(log_data['user_id'])

# Проверка корректности кодирования
print(log_data[['user_id', 'user_id_encoded', 'video_id', 'video_id_encoded']].head())


                                       user_id  user_id_encoded  \
878270    67364097-11ac-4efe-a716-00c7b6388ad3             3634   
33363377  84ba2477-abe7-4399-abcb-ba3fbe1d4b99             4681   
13472745  19ad77bf-1264-429e-8d95-87891f5bc9bd              933   
24159069  ba6d46f8-c2ea-4531-8ee2-9ddace35f24e             6534   
12069240  8fb2a78c-d48b-4035-80d0-5dfbd235cd07             5057   

                                      video_id  video_id_encoded  
878270    6e0efb88-e04c-4244-8e5d-880c87a1d74b            669635  
33363377  549be903-bfe7-4699-9320-70214743676c            514700  
13472745  f31f9451-fe62-4a39-a70e-903a000131ad           1477300  
24159069  2d097c02-afaa-4006-99de-d4abff4ed5dc            273865  
12069240  f7958100-4ad3-43d4-8fa4-f95cb383a119           1504393  


In [11]:
# Создание последовательностей для обучения
X, y = [], []
for user_id in log_data['user_id'].unique():
    user_logs = log_data[log_data['user_id'] == user_id]
    seq = user_logs['video_id_encoded'].tolist()
    for i in range(1, len(seq)):
        X.append(seq[:i])  # Все до текущего
        y.append(seq[i])   # Текущее видео

# Преобразование данных в массивы и ограничение длины последовательностей
max_seq_len = 50  # Ограничим длину последовательностей
X = [x[-max_seq_len:] for x in X]  # Ограничим длину последовательности
X = [np.pad(x, (max_seq_len - len(x), 0), mode='constant') for x in X]  # Паддинг последовательностей
X = np.array(X)
y = np.array(y)


In [12]:
# Разделение на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
# Создание класса Dataset для PyTorch
class VideoDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Создание DataLoader для загрузки данных батчами
train_dataset = VideoDataset(X_train, y_train)
val_dataset = VideoDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [14]:
# Определение модели
class VideoRecommendationModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=50, hidden_dim=100, output_dim=None):
        super(VideoRecommendationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        hidden = self.dropout(hidden[-1])
        out = self.fc(hidden)
        return out

# Инициализация модели, функции потерь и оптимизатора
vocab_size = len(video_encoder.classes_)
model = VideoRecommendationModel(vocab_size=vocab_size, output_dim=vocab_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [15]:
# Функция тренировки модели
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}")


In [16]:
# Запуск тренировки
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)


Epoch 1/5, Train Loss: 14.253737926483154, Val Loss: 14.268963813781738
Epoch 2/5, Train Loss: 14.186275005340576, Val Loss: 14.26468276977539
Epoch 3/5, Train Loss: 14.117308139801025, Val Loss: 14.258644104003906
Epoch 4/5, Train Loss: 14.03763723373413, Val Loss: 14.252567291259766
Epoch 5/5, Train Loss: 13.941096305847168, Val Loss: 14.24657917022705


In [17]:
import torch

# Путь для сохранения модели
model_save_path = 'video_recommendation_model.pth'

# Сохранение модели после тренировки
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, model_save_path)

print(f"Модель сохранена по пути: {model_save_path}")


Модель сохранена по пути: video_recommendation_model.pth


: 